In [1]:
from util import load_data, fdr, plot_report
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from collections import ChainMap

In [7]:
x_train, x_test, y_train, y_test = load_data('var_30.csv')
x_train.head()

,ssn_name_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,ssn_firstname_count_14,ssn_dob_count_0_by_30,name_dob_count_0_by_30,fulladdress_homephone_count_14,fulladdress_count_1_by_14,fulladdress_homephone_day_since,ssn_dob_day_since,...,address_count_0,name_count_7,ssn_count_0_by_14,fulladdress_count_1_by_7,homephone_count_7,homephone_count_3,ssn_count_3,ssn_lastname_day_since,ssn_count_7,ssn_firstname_count_7
352972,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.342218,-0.349123,...,-0.042158,-0.115648,0.124781,0.09144,0.055132,0.639149,-0.047591,-0.332021,-0.058092,-0.057185
121897,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-1.149174,-1.155577,...,-0.042158,-0.115648,0.124781,0.09144,-0.769214,-0.521454,-0.047591,-1.140309,-0.058092,-0.057185
570285,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,0.407776,0.400405,...,-0.042158,-0.115648,0.124781,0.09144,1.703825,1.799751,-0.047591,0.419211,-0.058092,-0.057185
723593,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,0.939417,0.931715,...,-0.042158,-0.115648,0.124781,0.09144,-0.769214,-0.521454,-0.047591,0.951729,-0.058092,-0.057185
120398,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-1.158668,-1.165064,...,-0.042158,-0.115648,0.124781,0.09144,0.879479,0.639149,-0.047591,-1.149818,-0.058092,-0.057185


In [ ]:
rf = RandomForestClassifier(verbose=1)
params = {'n_estimators': [300, 500, 700, 900],
          'max_depth': [80, 95, 110],
          'max_features':[6, 7, 8, 9]}
grid = GridSearchCV(rf, params, cv=2, refit='FDR', 
                    verbose=1, n_jobs=8,
                    scoring={'FDR': fdr})
grid.fit(x_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


In [5]:
print(f"The best parameters are {grid.best_params_} "
      f"with a score of {grid.best_score_:0.5f}")
pd.DataFrame(grid.cv_results_).filter(
    items=['param_n_estimators', 'param_class_weight',
           'param_max_depth', 'mean_test_FDR',
           'std_test_FDR', 'rank_test_FDR',
           'mean_test_ROC', 'std_test_ROC'])

The best parameters are {'max_depth': 80, 'max_features': 9, 'n_estimators': 700} with a score of 0.52092


,param_n_estimators,param_max_depth,mean_test_FDR,std_test_FDR,rank_test_FDR
0,300,80,0.519439,0.008385,42
1,500,80,0.520049,0.008477,18
2,700,80,0.519874,0.008147,26
3,900,80,0.520310,0.008445,8
4,300,80,0.519787,0.007523,35
5,500,80,0.519439,0.008038,42
6,700,80,0.520049,0.007916,18
7,900,80,0.519874,0.007952,31
8,300,80,0.519874,0.008697,26
9,500,80,0.520659,0.008697,2


In [ ]:
rf = RandomForestClassifier(n_estimators=1000, 
                            max_depth=20, 
                            max_features=9,
                            ccp_alpha=1e-7,
                            n_jobs=8).fit(x_train, y_train)
fdr(rf, x_test, y_test)

In [ ]:
fdr(rf, x_train, y_train)